WOW. This was one hell of a project. I felt like I learned a lot but I have much much more to learn.
Below is the code I used to fulfill the deliverables for Task 3 of the Fall 2019 Wedge Project, provided to us courtesy of Prof. John Chandler in the Applied Data Analytics course. You will find my 

In [ ]:
#Import stuff I may or may not need!

import datetime
import random
import os
import csv
import sqlite3

#imports for GBQ
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
#Code to connect to GBQ instance 

service_path = "C:\\Users\\Alexi\\Documents\\UofM\\Fall2019\\ADA\\applied-data-analytics\\wedge-investigation\\"
service_file = 'WedgeProject-6e342901e4a7.json'
gbq_proj_id = 'wedgeproject' 
gbq_dataset_id = 'WedgeData' 


# Pass in credentials to GBQ and establish connection with instance
credentials = service_account.Credentials.from_service_account_file("C:\\Users\\Alexi\\Documents\\UofM\\Fall2019\\ADA\\applied-data-analytics\\wedge-investigation\\WedgeProject-6e342901e4a7.json")
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

# TABLE 1

In [ ]:
#Here I connect to my local file, exported from GBQ, because I couldn't write
#a proper SQL statement in Python

input_file = "wedgeproject_3.1table.csv"

#write to file
with open(input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split(","))
        #if idx > 5 :
            #break

In [ ]:
#connect to SQLite and cursor
db = sqlite3.connect("wedgeproject_part3.db")
cur = db.cursor()

In [ ]:
#Create my table1 in SQLite 

cur.execute('''DROP TABLE IF EXISTS datetimebyhour''')
cur.execute('''CREATE TABLE datetimebyhour (
    dt TIMESTAMP,
    hour INTEGER,
    total_spend REAL,
    transactions INTEGER,
    total_items INTEGER)
''')


# fill the table
with open(input_file,'r', encoding="Latin-1") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO datetimebyhour (dt,hour,total_spend,
              transactions,total_items)
            VALUES (?,?,?,?,?)''', line)

        #if idx > 100 :
        #    break
        #feel free to remove these and comment out execution code if I want to see what's up
        #in the table without opening my SQLiteDB Browser.

db.commit()
db.close()

# TABLE 2

In [ ]:
#Here I connect to my local file, exported from GBQ, because I couldn't write
#a proper SQL statement in Python

input_file = "wedgeproject_3.2table.csv"


with open(input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split(","))
        if idx > 5 :
            break


db = sqlite3.connect("wedgeproject_part3.db")
cur = db.cursor()

In [ ]:
#Table 2 Query - Query in GBQ Console and exported. View saved queries on 10.24.2019 if curious.
#Create my table in SQLite 

cur.execute('''DROP TABLE IF EXISTS cardno_year_month''')
cur.execute('''CREATE TABLE cardno_year_month (
    card_no INTEGER,
    year INTEGER,
    month INTEGER,
    total_spend REAL,
    transactions INTEGER,
    total_items INTEGER)
''')


#and fill 'er on up'
with open(input_file,'r', encoding="Latin-1") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO cardno_year_month (card_no,year,month,
              total_spend,transactions,total_items)
            VALUES (?,?,?,?,?,?)''', line)


db.commit()
db.close()

# TABLE 3

In [ ]:
#Table 3 Query
query3 = '''SELECT
 EXTRACT(year
 FROM
   datetime) AS year,
 EXTRACT(month
 FROM
   datetime) AS month,
 upc,
 SUM(total) AS spend,
 COUNT(DISTINCT(CONCAT(CAST(EXTRACT(year
         FROM
           datetime) AS string), CAST(EXTRACT(month
         FROM
           datetime) AS string), CAST(EXTRACT(day
         FROM
           datetime) AS string), CAST(register_no AS string), 
                                   CAST(emp_no AS string), 
                                   CAST(trans_no AS string)))) AS Transactions,
 SUM(CASE
     WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
   ELSE
   1
 END
   ) AS Items,
 description,
 dept_name,
 trans.department
FROM
 `wedgeproject.WedgeData.transArchive_*` AS trans
JOIN
 `wedgeproject.WedgeData.wedge_dept_name_dept_no` dl
ON
 trans.department = dl.department
WHERE
 trans.department != 0
 AND trans.department != 15
 AND trans_status != 'M'
 AND trans_status != 'C'
 AND trans_status != 'J'
 AND (trans_status IS NULL
   OR trans_status = ' '
   OR trans_status = 'V'
   OR trans_status = 'R')
GROUP BY
 upc,
 description,
 dept_name,
 trans.department,
 year,
 month
ORDER BY
 year,
 month'''

In [ ]:
query_job = client.query(
    query3,
    location="US",
)
#fill table with data
with open("wedgeproject_3.3", 'w') as ofile :
    for idx, row in enumerate (query_job) :
        ofile.write(",".join([str(element) for element in row]) +"\n")

In [ ]:
#Here I connect to my local file, exported from GBQ, because I couldn't write
#a proper SQL statement in Python

input_file = "wedgeproject_3.3table.csv"

#write to file
with open(input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split(","))
        #if idx > 10 :
            #break


db = sqlite3.connect("wedgeproject_part3.db")
cur = db.cursor()

In [ ]:
#Create my table in SQLite 

cur.execute('''DROP TABLE IF EXISTS table3''')
cur.execute('''CREATE TABLE table3 (
    year INTEGER,
    month INTEGER,
    upc INTEGER,
    spend REAL,
    Transactions INTEGER,
    Items INTEGER,
    description TEXT,
    dept_name TEXT,
    department INTEGER)
''')



In [ ]:
# fill the table
with open(input_file,'r', encoding="Latin-1") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO table3 (upc,year,month,spend,transactions,items,description,dept_name,department)
            VALUES (?,?,?,?,?,?,?,?,?)''', line)

        #if idx > 100 :
        #    break

db.commit()
db.close()